In [9]:
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error 
import sklearn.metrics as sklm
import math

In [10]:
from sklearn.metrics import mean_absolute_percentage_error 
import sklearn.metrics as sklm
import math
def print_metrics(y_true, y_predicted):
    ## First compute R^2 and the adjusted R^2
    r2 = sklm.r2_score(y_true, y_predicted)
    
    ## Print the usual metrics and the R^2 values
    print('MAPE                   = ' + str(mean_absolute_percentage_error(y_true, y_predicted)))
    print('Root Mean Square Error = ' + str(math.sqrt(sklm.mean_squared_error(y_true, y_predicted))))
    print('Mean Absolute Error    = ' + str(sklm.mean_absolute_error(y_true, y_predicted)))
    print('R^2                    = ' + str(r2))

In [11]:
df = pd.read_csv('data v_1.csv') 
df = df.drop(columns = 'Unnamed: 0')
df = df[df['Linear Feet'] >= 1]
df = df[df['Carrier Cost (Mixed)'] >= 300]
df = df[df['Carrier Cost (Mixed)'] <= df['Target buy rate']]
mapeo = {'Connecticut': 'Northeast', 'Delaware': 'Northeast', 'Maine': 'Northeast', 'Massachusetts': 'Northeast', 'Maryland': 'Northeast', 
         'New Hampshire': 'Northeast', 'New Jersey': 'Northeast', 'New York': 'Northeast', 'Pennsylvania': 'Northeast', 'Rhode Island': 'Northeast', 
         'Vermont': 'Northeast', 'District of Columbia': 'Northeast', 
         'Alabama': 'Southeast', 'Arkansas': 'Southeast', 'Florida': 'Southeast', 'Georgia': 'Southeast', 
         'Kentucky': 'Southeast', 'Louisiana': 'Southeast', 'Mississippi': 'Southeast', 'North Carolina': 'Southeast', 
         'South Carolina': 'Southeast', 'Tennessee': 'Southeast', 'Virginia': 'Southeast', 'West Virginia': 'Southeast', 
         'Arizona': 'Southwest', 'Colorado': 'Southwest', 'Kansas': 'Southwest', 'New Mexico': 'Southwest', 
         'Oklahoma': 'Southwest', 'Texas': 'Southwest', 
         'Illinois': 'Midwest', 'Indiana': 'Midwest', 'Iowa': 'Midwest', 'Michigan': 'Midwest', 
         'Minnesota': 'Midwest', 'Missouri': 'Midwest', 'Nebraska': 'Midwest', 'North Dakota': 'Midwest', 
         'Ohio': 'Midwest', 'South Dakota': 'Midwest', 'Wisconsin': 'Midwest', 
         'California': 'West', 'Idaho': 'West', 'Montana': 'West', 'Nevada': 'West',
         'Oregon': 'West', 'Utah': 'West', 'Washington': 'West', 'Wyoming': 'West',
         'Ontario': 'Canada', 'British Columbia': 'Canada', 'Alberta': 'Canada', 'Quebec': 'Canada', 
         'Manitoba': 'Canada', 'Saskatchewan': 'Canada', 'Nova Scotia': 'Canada', 'New Brunswick': 'Canada', 
         'Newfoundland and Labrador': 'Canada', 'Yukon': 'Canada', 'Prince Edward Island': 'Canada', 'Northwest Territories': 'Canada', 
         'Alaska': 'Offshore'}
df['Region Origen'] = df['Origin State'].map(mapeo)
df['Region Destino'] = df['Destination State'].map(mapeo)
df.head(1)

,Load: Load Number,Load: Created Date,Actual Pickup Date,Origin Zip Code,Destination Zip Code,Origin State,Destination State,Mode,Linear Feet,Total Weight,Distance (Miles),Total HU,Revenue (Mixed) Currency,Revenue (Mixed),Carrier Cost (Mixed) Currency,Carrier Cost (Mixed),Target buy rate,GS Network buy rate,Region Origen,Region Destino
0,SHP5161932,8/5/2024,8/6/2024,45786,33316,Ohio,Florida,Partial F/SD (LTL Spot Market),20.5,1675.0,1081.85,2.0,USD,1600.0,USD,1450.0,2761.0,2732.0,Midwest,Southeast


In [12]:
#Identificamos y creamos los grupos por estados
# Contar cuántas veces aparece cada estado como origen
origen_counts = df['Origin State'].value_counts().reset_index()
origen_counts.columns = ['Estado', 'Cuenta_Origen']

# Contar cuántas veces aparece cada estado como destino
destino_counts = df['Destination State'].value_counts().reset_index()
destino_counts.columns = ['Estado', 'Cuenta_Destino']

# Combinar las cuentas de origen y destino
counts = pd.merge(origen_counts, destino_counts, on='Estado', how='outer').fillna(0)

# Añadir la información al DataFrame original
df = pd.merge(df, counts, left_on='Origin State', right_on='Estado', how='left').drop(columns='Estado')
df = pd.merge(df, counts, left_on='Destination State', right_on='Estado', how='left', suffixes=('_Origen', '_Destino')).drop(columns='Estado')

df = df.drop(columns=['Cuenta_Destino_Origen', 'Cuenta_Origen_Destino'])
df.head(2)

,Load: Load Number,Load: Created Date,Actual Pickup Date,Origin Zip Code,Destination Zip Code,Origin State,Destination State,Mode,Linear Feet,Total Weight,...,Revenue (Mixed) Currency,Revenue (Mixed),Carrier Cost (Mixed) Currency,Carrier Cost (Mixed),Target buy rate,GS Network buy rate,Region Origen,Region Destino,Cuenta_Origen_Origen,Cuenta_Destino_Destino
0,SHP5161932,8/5/2024,8/6/2024,45786,33316,Ohio,Florida,Partial F/SD (LTL Spot Market),20.5,1675.0,...,USD,1600.00,USD,1450.0,2761.0,2732.0,Midwest,Southeast,77.0,1061.0
1,SHP5159992,8/2/2024,8/5/2024,63068,84404,Missouri,Utah,Partial Van (LTL Spot Market),14.0,3800.0,...,USD,1839.56,USD,1400.0,2640.0,2618.0,Midwest,West,33.0,12.0


In [13]:
df['Cuenta_Origen_Origen'] = df.apply(
    lambda row: 'California' if row['Origin State'] == 'California' else row['Cuenta_Origen_Origen'],
    axis=1
)
df['Cuenta_Destino_Destino'] = df.apply(
    lambda row: 'California' if row['Destination State'] == 'California' else row['Cuenta_Destino_Destino'],
    axis=1
)
df['Cuenta_Origen_Origen'] = df.apply(
    lambda row: 'Florida' if row['Origin State'] == 'Florida' else row['Cuenta_Origen_Origen'],
    axis=1
)
df['Cuenta_Destino_Destino'] = df.apply(
    lambda row: 'Florida' if row['Destination State'] == 'Florida' else row['Cuenta_Destino_Destino'],
    axis=1
)
df['Cuenta_Origen_Origen'] = df.apply(
    lambda row: 'Texas' if row['Origin State'] == 'Texas' else row['Cuenta_Origen_Origen'],
    axis=1
)
df['Cuenta_Destino_Destino'] = df.apply(
    lambda row: 'Texas' if row['Destination State'] == 'Texas' else row['Cuenta_Destino_Destino'],
    axis=1
)
df['Cuenta_Origen_Origen'] = df.apply(
    lambda row: 'Illinois' if row['Origin State'] == 'Illinois' else row['Cuenta_Origen_Origen'],
    axis=1
)
df['Cuenta_Destino_Destino'] = df.apply(
    lambda row: 'Illinois' if row['Destination State'] == 'Illinois' else row['Cuenta_Destino_Destino'],
    axis=1
)
df.head(2)

,Load: Load Number,Load: Created Date,Actual Pickup Date,Origin Zip Code,Destination Zip Code,Origin State,Destination State,Mode,Linear Feet,Total Weight,...,Revenue (Mixed) Currency,Revenue (Mixed),Carrier Cost (Mixed) Currency,Carrier Cost (Mixed),Target buy rate,GS Network buy rate,Region Origen,Region Destino,Cuenta_Origen_Origen,Cuenta_Destino_Destino
0,SHP5161932,8/5/2024,8/6/2024,45786,33316,Ohio,Florida,Partial F/SD (LTL Spot Market),20.5,1675.0,...,USD,1600.00,USD,1450.0,2761.0,2732.0,Midwest,Southeast,77.0,Florida
1,SHP5159992,8/2/2024,8/5/2024,63068,84404,Missouri,Utah,Partial Van (LTL Spot Market),14.0,3800.0,...,USD,1839.56,USD,1400.0,2640.0,2618.0,Midwest,West,33.0,12.0


In [14]:
# Definir el umbral
umbral = 25

# Función para transformar valores
def transformar_valor(x):
    if isinstance(x, (int, float)):  # Verificar si es un número
        return 'Intermedio' if x > umbral else 'Dificil'
    else:
        return x  # Mantener valores categóricos

# Aplicar la función a la columna
df['Cuenta_Origen_Origen'] = df['Cuenta_Origen_Origen'].apply(transformar_valor)
df['Cuenta_Destino_Destino'] = df['Cuenta_Destino_Destino'].apply(transformar_valor)

In [15]:
df.head(1)

,Load: Load Number,Load: Created Date,Actual Pickup Date,Origin Zip Code,Destination Zip Code,Origin State,Destination State,Mode,Linear Feet,Total Weight,...,Revenue (Mixed) Currency,Revenue (Mixed),Carrier Cost (Mixed) Currency,Carrier Cost (Mixed),Target buy rate,GS Network buy rate,Region Origen,Region Destino,Cuenta_Origen_Origen,Cuenta_Destino_Destino
0,SHP5161932,8/5/2024,8/6/2024,45786,33316,Ohio,Florida,Partial F/SD (LTL Spot Market),20.5,1675.0,...,USD,1600.0,USD,1450.0,2761.0,2732.0,Midwest,Southeast,Intermedio,Florida


In [16]:
df['Combinacion estados'] = df['Cuenta_Origen_Origen'] + df['Cuenta_Destino_Destino'] 
df['Combinacion regiones'] = df['Region Origen'] + df['Region Destino'] 
df.head(1)

,Load: Load Number,Load: Created Date,Actual Pickup Date,Origin Zip Code,Destination Zip Code,Origin State,Destination State,Mode,Linear Feet,Total Weight,...,Carrier Cost (Mixed) Currency,Carrier Cost (Mixed),Target buy rate,GS Network buy rate,Region Origen,Region Destino,Cuenta_Origen_Origen,Cuenta_Destino_Destino,Combinacion estados,Combinacion regiones
0,SHP5161932,8/5/2024,8/6/2024,45786,33316,Ohio,Florida,Partial F/SD (LTL Spot Market),20.5,1675.0,...,USD,1450.0,2761.0,2732.0,Midwest,Southeast,Intermedio,Florida,IntermedioFlorida,MidwestSoutheast


### Todas las variables

In [9]:
#Creamos un dataframe solo con las variables de nuestro interes
columnas_seleccionadas = ['Linear Feet', 'Total Weight', 'Distance (Miles)', 'Target buy rate', 
                          'Cuenta_Origen_Origen', 'Cuenta_Destino_Destino', 'Region Origen', 'Region Destino',
                          'Combinacion estados', 'Combinacion regiones', 'Carrier Cost (Mixed)']
df_nuevo = df[columnas_seleccionadas]

In [10]:
#Separamos la data en test y train (20-80)
from sklearn.model_selection import train_test_split

X = df_nuevo[['Linear Feet', 'Total Weight', 'Distance (Miles)', 'Target buy rate', 
        'Cuenta_Origen_Origen', 'Cuenta_Destino_Destino', 'Region Origen', 'Region Destino',
        'Combinacion estados', 'Combinacion regiones',]]
y = df_nuevo['Carrier Cost (Mixed)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
#Prueba con catboost 
from catboost import CatBoostRegressor

# Crear el modelo
model = CatBoostRegressor(depth=5, learning_rate=0.025)

# Ajustar el modelo (especificando las columnas categóricas con el parámetro cat_features)
model.fit(X_train, y_train, cat_features=[4, 5, 6, 7, 8, 9])  # cat_features indica los índices de las variables categóricas

# Hacer predicciones
predictions = model.predict(X_test)

0:	learn: 644.5550891	total: 161ms	remaining: 2m 41s
1:	learn: 636.4690081	total: 182ms	remaining: 1m 30s
2:	learn: 628.9148534	total: 203ms	remaining: 1m 7s
3:	learn: 621.4146840	total: 222ms	remaining: 55.4s
4:	learn: 614.1324247	total: 237ms	remaining: 47.2s
5:	learn: 607.3990439	total: 257ms	remaining: 42.6s
6:	learn: 600.9995679	total: 276ms	remaining: 39.2s
7:	learn: 594.4220549	total: 296ms	remaining: 36.7s
8:	learn: 588.1990137	total: 315ms	remaining: 34.7s
9:	learn: 582.3567314	total: 334ms	remaining: 33.1s
10:	learn: 577.0464021	total: 354ms	remaining: 31.8s
11:	learn: 571.0245207	total: 381ms	remaining: 31.4s
12:	learn: 566.2037150	total: 397ms	remaining: 30.1s
13:	learn: 560.6978365	total: 420ms	remaining: 29.5s
14:	learn: 555.1813665	total: 441ms	remaining: 29s
15:	learn: 550.7089091	total: 455ms	remaining: 28s
16:	learn: 545.5653032	total: 481ms	remaining: 27.8s
17:	learn: 540.8643593	total: 508ms	remaining: 27.7s
18:	learn: 536.3814237	total: 530ms	remaining: 27.3s
19:	l

160:	learn: 351.4667761	total: 3.33s	remaining: 17.4s
161:	learn: 351.2972337	total: 3.35s	remaining: 17.4s
162:	learn: 350.8110636	total: 3.38s	remaining: 17.3s
163:	learn: 350.7119577	total: 3.39s	remaining: 17.3s
164:	learn: 350.3067979	total: 3.41s	remaining: 17.3s
165:	learn: 350.1674157	total: 3.43s	remaining: 17.2s
166:	learn: 349.7305863	total: 3.45s	remaining: 17.2s
167:	learn: 349.6237933	total: 3.47s	remaining: 17.2s
168:	learn: 349.2808277	total: 3.49s	remaining: 17.1s
169:	learn: 349.0275133	total: 3.5s	remaining: 17.1s
170:	learn: 348.9018158	total: 3.52s	remaining: 17.1s
171:	learn: 348.9015049	total: 3.53s	remaining: 17s
172:	learn: 348.5115927	total: 3.55s	remaining: 17s
173:	learn: 348.2276854	total: 3.57s	remaining: 17s
174:	learn: 348.1387942	total: 3.59s	remaining: 16.9s
175:	learn: 348.0499258	total: 3.61s	remaining: 16.9s
176:	learn: 347.6603839	total: 3.63s	remaining: 16.9s
177:	learn: 347.5641059	total: 3.64s	remaining: 16.8s
178:	learn: 347.3104485	total: 3.66

317:	learn: 324.8621688	total: 6.29s	remaining: 13.5s
318:	learn: 324.6450926	total: 6.32s	remaining: 13.5s
319:	learn: 324.5066342	total: 6.33s	remaining: 13.5s
320:	learn: 324.4235345	total: 6.35s	remaining: 13.4s
321:	learn: 324.4161459	total: 6.37s	remaining: 13.4s
322:	learn: 324.4103117	total: 6.39s	remaining: 13.4s
323:	learn: 324.3084163	total: 6.41s	remaining: 13.4s
324:	learn: 323.9040148	total: 6.42s	remaining: 13.3s
325:	learn: 323.7974519	total: 6.44s	remaining: 13.3s
326:	learn: 323.7123447	total: 6.46s	remaining: 13.3s
327:	learn: 323.5823938	total: 6.48s	remaining: 13.3s
328:	learn: 323.5051519	total: 6.5s	remaining: 13.3s
329:	learn: 323.4250006	total: 6.52s	remaining: 13.2s
330:	learn: 323.1630424	total: 6.54s	remaining: 13.2s
331:	learn: 322.9434116	total: 6.56s	remaining: 13.2s
332:	learn: 322.5528061	total: 6.58s	remaining: 13.2s
333:	learn: 322.4773821	total: 6.59s	remaining: 13.2s
334:	learn: 322.3543333	total: 6.61s	remaining: 13.1s
335:	learn: 322.2668030	total

476:	learn: 307.4311914	total: 9.41s	remaining: 10.3s
477:	learn: 307.3395781	total: 9.43s	remaining: 10.3s
478:	learn: 307.3084053	total: 9.45s	remaining: 10.3s
479:	learn: 307.2647283	total: 9.47s	remaining: 10.3s
480:	learn: 307.1794113	total: 9.49s	remaining: 10.2s
481:	learn: 307.1388153	total: 9.51s	remaining: 10.2s
482:	learn: 307.0837936	total: 9.53s	remaining: 10.2s
483:	learn: 307.0421067	total: 9.55s	remaining: 10.2s
484:	learn: 306.9218461	total: 9.57s	remaining: 10.2s
485:	learn: 306.6562524	total: 9.59s	remaining: 10.1s
486:	learn: 306.5983690	total: 9.61s	remaining: 10.1s
487:	learn: 306.5154379	total: 9.63s	remaining: 10.1s
488:	learn: 306.4344242	total: 9.65s	remaining: 10.1s
489:	learn: 306.2416906	total: 9.67s	remaining: 10.1s
490:	learn: 306.1484064	total: 9.69s	remaining: 10s
491:	learn: 305.8162625	total: 9.71s	remaining: 10s
492:	learn: 305.6028566	total: 9.72s	remaining: 10s
493:	learn: 305.4471748	total: 9.74s	remaining: 9.98s
494:	learn: 305.2999755	total: 9.7

638:	learn: 293.1413685	total: 12.6s	remaining: 7.11s
639:	learn: 292.9693154	total: 12.6s	remaining: 7.09s
640:	learn: 292.9276222	total: 12.6s	remaining: 7.07s
641:	learn: 292.9000974	total: 12.6s	remaining: 7.05s
642:	learn: 292.8721877	total: 12.7s	remaining: 7.03s
643:	learn: 292.8443242	total: 12.7s	remaining: 7.01s
644:	learn: 292.7712326	total: 12.7s	remaining: 6.99s
645:	learn: 292.5882932	total: 12.7s	remaining: 6.97s
646:	learn: 292.5805176	total: 12.7s	remaining: 6.95s
647:	learn: 292.5284632	total: 12.8s	remaining: 6.93s
648:	learn: 292.4936996	total: 12.8s	remaining: 6.91s
649:	learn: 292.4520196	total: 12.8s	remaining: 6.89s
650:	learn: 292.3498843	total: 12.8s	remaining: 6.87s
651:	learn: 292.2916771	total: 12.8s	remaining: 6.85s
652:	learn: 292.2775475	total: 12.9s	remaining: 6.83s
653:	learn: 292.2031539	total: 12.9s	remaining: 6.81s
654:	learn: 292.1623903	total: 12.9s	remaining: 6.79s
655:	learn: 292.1321580	total: 12.9s	remaining: 6.77s
656:	learn: 291.9858727	tota

799:	learn: 283.6189726	total: 15.7s	remaining: 3.93s
800:	learn: 283.5887505	total: 15.7s	remaining: 3.91s
801:	learn: 283.5218480	total: 15.8s	remaining: 3.89s
802:	learn: 283.3629093	total: 15.8s	remaining: 3.87s
803:	learn: 283.2400303	total: 15.8s	remaining: 3.85s
804:	learn: 283.1203209	total: 15.8s	remaining: 3.83s
805:	learn: 283.0491217	total: 15.8s	remaining: 3.81s
806:	learn: 283.0491124	total: 15.9s	remaining: 3.79s
807:	learn: 283.0017002	total: 15.9s	remaining: 3.77s
808:	learn: 282.9882661	total: 15.9s	remaining: 3.75s
809:	learn: 282.9493137	total: 15.9s	remaining: 3.73s
810:	learn: 282.8630260	total: 15.9s	remaining: 3.71s
811:	learn: 282.7835965	total: 16s	remaining: 3.69s
812:	learn: 282.7199185	total: 16s	remaining: 3.68s
813:	learn: 282.5630706	total: 16s	remaining: 3.66s
814:	learn: 282.4479738	total: 16s	remaining: 3.64s
815:	learn: 282.4263052	total: 16.1s	remaining: 3.62s
816:	learn: 282.3568609	total: 16.1s	remaining: 3.6s
817:	learn: 282.1693246	total: 16.1s	

953:	learn: 275.6738436	total: 18.8s	remaining: 905ms
954:	learn: 275.5413824	total: 18.8s	remaining: 886ms
955:	learn: 275.4813587	total: 18.8s	remaining: 866ms
956:	learn: 275.4640290	total: 18.8s	remaining: 846ms
957:	learn: 275.4478736	total: 18.8s	remaining: 826ms
958:	learn: 275.4387893	total: 18.9s	remaining: 807ms
959:	learn: 275.2637668	total: 18.9s	remaining: 787ms
960:	learn: 275.2036090	total: 18.9s	remaining: 767ms
961:	learn: 275.1737664	total: 18.9s	remaining: 748ms
962:	learn: 275.0374283	total: 18.9s	remaining: 728ms
963:	learn: 275.0203690	total: 19s	remaining: 708ms
964:	learn: 275.0101927	total: 19s	remaining: 689ms
965:	learn: 274.9780116	total: 19s	remaining: 669ms
966:	learn: 274.8332423	total: 19s	remaining: 649ms
967:	learn: 274.7573822	total: 19s	remaining: 630ms
968:	learn: 274.7335091	total: 19.1s	remaining: 610ms
969:	learn: 274.7308183	total: 19.1s	remaining: 590ms
970:	learn: 274.6653581	total: 19.1s	remaining: 571ms
971:	learn: 274.6554478	total: 19.1s	r

In [12]:
print_metrics(y_test, predictions)

MAPE                   = 0.1803527063538102
Root Mean Square Error = 424.42680515355454
Mean Absolute Error    = 273.0588584045227
R^2                    = 0.6053462466611038


In [13]:
var = model.feature_importances_
for i in range(0, len(var)):
    print(X.columns[i], var[i])


Linear Feet 30.364571232299074
Total Weight 11.967095578457597
Distance (Miles) 4.074658798576075
Target buy rate 44.86999415254001
Cuenta_Origen_Origen 2.2301266468915553
Cuenta_Destino_Destino 1.1396216411406308
Region Origen 1.4183727536980872
Region Destino 1.0013652031336389
Combinacion estados 1.2390026910574157
Combinacion regiones 1.6951913022059402


### Solo estados

In [14]:
#Creamos un dataframe solo con las variables de nuestro interes
columnas_seleccionadas = ['Linear Feet', 'Total Weight', 'Distance (Miles)', 'Target buy rate', 
                          'Cuenta_Origen_Origen', 'Cuenta_Destino_Destino', 'Combinacion estados','Carrier Cost (Mixed)']
df_nuevo = df[columnas_seleccionadas]

In [15]:
#Separamos la data en test y train (20-80)
from sklearn.model_selection import train_test_split

X = df_nuevo[['Linear Feet', 'Total Weight', 'Distance (Miles)', 'Target buy rate', 
        'Cuenta_Origen_Origen', 'Cuenta_Destino_Destino', 'Combinacion estados']]
y = df_nuevo['Carrier Cost (Mixed)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
#Prueba con catboost 
from catboost import CatBoostRegressor

# Crear el modelo
model = CatBoostRegressor(depth=5, learning_rate=0.025)

# Ajustar el modelo (especificando las columnas categóricas con el parámetro cat_features)
model.fit(X_train, y_train, cat_features=[4, 5, 6])  # cat_features indica los índices de las variables categóricas

# Hacer predicciones
predictions = model.predict(X_test)

0:	learn: 644.2927182	total: 16.8ms	remaining: 16.8s
1:	learn: 636.5835382	total: 35.1ms	remaining: 17.5s
2:	learn: 629.4468260	total: 52.5ms	remaining: 17.4s
3:	learn: 622.2522771	total: 72ms	remaining: 17.9s
4:	learn: 614.3079052	total: 92.4ms	remaining: 18.4s
5:	learn: 607.4667080	total: 110ms	remaining: 18.3s
6:	learn: 600.4019224	total: 128ms	remaining: 18.2s
7:	learn: 593.7835250	total: 142ms	remaining: 17.7s
8:	learn: 588.2034544	total: 157ms	remaining: 17.3s
9:	learn: 581.7162606	total: 176ms	remaining: 17.4s
10:	learn: 575.5860491	total: 193ms	remaining: 17.3s
11:	learn: 569.6116902	total: 211ms	remaining: 17.4s
12:	learn: 564.0979133	total: 231ms	remaining: 17.5s
13:	learn: 558.6490391	total: 251ms	remaining: 17.7s
14:	learn: 553.3235180	total: 268ms	remaining: 17.6s
15:	learn: 547.1876698	total: 286ms	remaining: 17.6s
16:	learn: 542.4143537	total: 303ms	remaining: 17.5s
17:	learn: 536.9460926	total: 321ms	remaining: 17.5s
18:	learn: 532.3470014	total: 338ms	remaining: 17.5s


156:	learn: 348.3280941	total: 3.14s	remaining: 16.9s
157:	learn: 347.9687708	total: 3.16s	remaining: 16.9s
158:	learn: 347.7486203	total: 3.19s	remaining: 16.8s
159:	learn: 347.6057778	total: 3.2s	remaining: 16.8s
160:	learn: 347.5185399	total: 3.21s	remaining: 16.8s
161:	learn: 347.3896368	total: 3.23s	remaining: 16.7s
162:	learn: 347.0878219	total: 3.25s	remaining: 16.7s
163:	learn: 346.8244833	total: 3.27s	remaining: 16.7s
164:	learn: 346.4982389	total: 3.29s	remaining: 16.7s
165:	learn: 346.2481269	total: 3.32s	remaining: 16.7s
166:	learn: 346.0823552	total: 3.35s	remaining: 16.7s
167:	learn: 345.9204785	total: 3.37s	remaining: 16.7s
168:	learn: 345.4322058	total: 3.4s	remaining: 16.7s
169:	learn: 345.1900173	total: 3.42s	remaining: 16.7s
170:	learn: 345.0085131	total: 3.44s	remaining: 16.7s
171:	learn: 344.7369627	total: 3.46s	remaining: 16.7s
172:	learn: 344.4174294	total: 3.48s	remaining: 16.6s
173:	learn: 344.3399381	total: 3.49s	remaining: 16.6s
174:	learn: 344.0677827	total:

315:	learn: 321.2193979	total: 6.14s	remaining: 13.3s
316:	learn: 321.1499795	total: 6.17s	remaining: 13.3s
317:	learn: 320.9199896	total: 6.19s	remaining: 13.3s
318:	learn: 320.8442121	total: 6.21s	remaining: 13.3s
319:	learn: 320.8246869	total: 6.23s	remaining: 13.2s
320:	learn: 320.6581239	total: 6.25s	remaining: 13.2s
321:	learn: 320.5401582	total: 6.27s	remaining: 13.2s
322:	learn: 320.5016507	total: 6.3s	remaining: 13.2s
323:	learn: 320.4283585	total: 6.32s	remaining: 13.2s
324:	learn: 320.3536176	total: 6.33s	remaining: 13.2s
325:	learn: 320.2870156	total: 6.36s	remaining: 13.1s
326:	learn: 320.0920940	total: 6.38s	remaining: 13.1s
327:	learn: 319.8496865	total: 6.4s	remaining: 13.1s
328:	learn: 319.8250090	total: 6.42s	remaining: 13.1s
329:	learn: 319.7665637	total: 6.44s	remaining: 13.1s
330:	learn: 319.7051479	total: 6.46s	remaining: 13.1s
331:	learn: 319.4355501	total: 6.48s	remaining: 13s
332:	learn: 319.4062895	total: 6.5s	remaining: 13s
333:	learn: 319.3696705	total: 6.51

472:	learn: 303.9036148	total: 9.23s	remaining: 10.3s
473:	learn: 303.8381382	total: 9.25s	remaining: 10.3s
474:	learn: 303.7966586	total: 9.27s	remaining: 10.3s
475:	learn: 303.7645099	total: 9.3s	remaining: 10.2s
476:	learn: 303.7109963	total: 9.32s	remaining: 10.2s
477:	learn: 303.6735902	total: 9.34s	remaining: 10.2s
478:	learn: 303.5903893	total: 9.35s	remaining: 10.2s
479:	learn: 303.4855884	total: 9.37s	remaining: 10.2s
480:	learn: 303.4336396	total: 9.39s	remaining: 10.1s
481:	learn: 303.2303938	total: 9.41s	remaining: 10.1s
482:	learn: 303.1587884	total: 9.43s	remaining: 10.1s
483:	learn: 303.0554395	total: 9.45s	remaining: 10.1s
484:	learn: 302.9428004	total: 9.47s	remaining: 10.1s
485:	learn: 302.8717493	total: 9.48s	remaining: 10s
486:	learn: 302.7395122	total: 9.5s	remaining: 10s
487:	learn: 302.6164974	total: 9.52s	remaining: 9.99s
488:	learn: 302.4986988	total: 9.54s	remaining: 9.97s
489:	learn: 302.4181283	total: 9.55s	remaining: 9.95s
490:	learn: 302.4103364	total: 9.5

631:	learn: 289.5861202	total: 12.2s	remaining: 7.07s
632:	learn: 289.5344933	total: 12.2s	remaining: 7.05s
633:	learn: 289.4558593	total: 12.2s	remaining: 7.04s
634:	learn: 289.4228077	total: 12.2s	remaining: 7.01s
635:	learn: 289.3533425	total: 12.2s	remaining: 6.99s
636:	learn: 289.2873489	total: 12.2s	remaining: 6.97s
637:	learn: 289.1793645	total: 12.3s	remaining: 6.95s
638:	learn: 289.0573621	total: 12.3s	remaining: 6.93s
639:	learn: 288.8633196	total: 12.3s	remaining: 6.91s
640:	learn: 288.7994266	total: 12.3s	remaining: 6.9s
641:	learn: 288.6959493	total: 12.3s	remaining: 6.88s
642:	learn: 288.6397707	total: 12.4s	remaining: 6.86s
643:	learn: 288.5944993	total: 12.4s	remaining: 6.84s
644:	learn: 288.5519104	total: 12.4s	remaining: 6.82s
645:	learn: 288.4683901	total: 12.4s	remaining: 6.8s
646:	learn: 288.3468263	total: 12.4s	remaining: 6.78s
647:	learn: 288.3281524	total: 12.4s	remaining: 6.76s
648:	learn: 288.2598976	total: 12.5s	remaining: 6.74s
649:	learn: 288.1829021	total:

790:	learn: 278.2940817	total: 15.1s	remaining: 3.98s
791:	learn: 278.2460547	total: 15.1s	remaining: 3.96s
792:	learn: 278.2162070	total: 15.1s	remaining: 3.94s
793:	learn: 278.1650732	total: 15.1s	remaining: 3.92s
794:	learn: 278.1195257	total: 15.1s	remaining: 3.9s
795:	learn: 277.9633883	total: 15.2s	remaining: 3.88s
796:	learn: 277.7883889	total: 15.2s	remaining: 3.86s
797:	learn: 277.7726257	total: 15.2s	remaining: 3.84s
798:	learn: 277.7154804	total: 15.2s	remaining: 3.83s
799:	learn: 277.6639071	total: 15.2s	remaining: 3.81s
800:	learn: 277.5595903	total: 15.2s	remaining: 3.79s
801:	learn: 277.4586444	total: 15.3s	remaining: 3.77s
802:	learn: 277.4233209	total: 15.3s	remaining: 3.75s
803:	learn: 277.3515639	total: 15.3s	remaining: 3.73s
804:	learn: 277.3288801	total: 15.3s	remaining: 3.71s
805:	learn: 277.2792252	total: 15.3s	remaining: 3.69s
806:	learn: 277.2360086	total: 15.3s	remaining: 3.67s
807:	learn: 277.1563349	total: 15.4s	remaining: 3.65s
808:	learn: 277.0920285	total

951:	learn: 268.2430734	total: 18s	remaining: 908ms
952:	learn: 268.2130735	total: 18s	remaining: 889ms
953:	learn: 268.2020630	total: 18.1s	remaining: 870ms
954:	learn: 268.1112647	total: 18.1s	remaining: 851ms
955:	learn: 268.0413901	total: 18.1s	remaining: 832ms
956:	learn: 267.8856692	total: 18.1s	remaining: 813ms
957:	learn: 267.8153680	total: 18.1s	remaining: 794ms
958:	learn: 267.7235319	total: 18.1s	remaining: 775ms
959:	learn: 267.7002437	total: 18.2s	remaining: 756ms
960:	learn: 267.6467216	total: 18.2s	remaining: 737ms
961:	learn: 267.5477773	total: 18.2s	remaining: 718ms
962:	learn: 267.4753636	total: 18.2s	remaining: 700ms
963:	learn: 267.4238923	total: 18.2s	remaining: 681ms
964:	learn: 267.3627266	total: 18.2s	remaining: 662ms
965:	learn: 267.3398038	total: 18.3s	remaining: 643ms
966:	learn: 267.3176919	total: 18.3s	remaining: 624ms
967:	learn: 267.2421195	total: 18.3s	remaining: 605ms
968:	learn: 267.2148093	total: 18.3s	remaining: 586ms
969:	learn: 267.1923350	total: 1

In [17]:
print_metrics(y_test, predictions)

MAPE                   = 0.17956740116737605
Root Mean Square Error = 421.2695165607078
Mean Absolute Error    = 272.9547149983813
R^2                    = 0.6111960242021551


In [18]:
var = model.feature_importances_
for i in range(0, len(var)):
    print(X.columns[i], var[i])


Linear Feet 30.100621652985172
Total Weight 12.128599840693719
Distance (Miles) 4.87170564756383
Target buy rate 46.07261719700812
Cuenta_Origen_Origen 2.9609499738145213
Cuenta_Destino_Destino 1.976307428248846
Combinacion estados 1.889198259685935


### Solo región

In [19]:
#Creamos un dataframe solo con las variables de nuestro interes
columnas_seleccionadas = ['Linear Feet', 'Total Weight', 'Distance (Miles)', 'Target buy rate', 
                          'Region Origen', 'Region Destino', 'Combinacion regiones','Carrier Cost (Mixed)']
df_nuevo = df[columnas_seleccionadas]

In [20]:
#Separamos la data en test y train (20-80)
from sklearn.model_selection import train_test_split

X = df_nuevo[['Linear Feet', 'Total Weight', 'Distance (Miles)', 'Target buy rate', 
        'Region Origen', 'Region Destino', 'Combinacion regiones']]
y = df_nuevo['Carrier Cost (Mixed)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
#Prueba con catboost 
from catboost import CatBoostRegressor

# Crear el modelo
model = CatBoostRegressor(depth=5, learning_rate=0.025)

# Ajustar el modelo (especificando las columnas categóricas con el parámetro cat_features)
model.fit(X_train, y_train, cat_features=[4, 5, 6])  # cat_features indica los índices de las variables categóricas

# Hacer predicciones
predictions = model.predict(X_test)

0:	learn: 644.2635648	total: 17.8ms	remaining: 17.8s
1:	learn: 636.6405825	total: 36.2ms	remaining: 18.1s
2:	learn: 629.1564640	total: 54.3ms	remaining: 18s
3:	learn: 621.1931733	total: 72.2ms	remaining: 18s
4:	learn: 614.2922188	total: 85.9ms	remaining: 17.1s
5:	learn: 606.8804275	total: 104ms	remaining: 17.3s
6:	learn: 599.1225186	total: 122ms	remaining: 17.3s
7:	learn: 591.9566716	total: 140ms	remaining: 17.4s
8:	learn: 585.6309868	total: 157ms	remaining: 17.3s
9:	learn: 578.8469514	total: 174ms	remaining: 17.2s
10:	learn: 573.3986524	total: 188ms	remaining: 16.9s
11:	learn: 566.7136035	total: 206ms	remaining: 17s
12:	learn: 560.9544649	total: 227ms	remaining: 17.3s
13:	learn: 556.4550343	total: 238ms	remaining: 16.8s
14:	learn: 551.7071656	total: 255ms	remaining: 16.8s
15:	learn: 545.7920266	total: 273ms	remaining: 16.8s
16:	learn: 540.4698840	total: 291ms	remaining: 16.8s
17:	learn: 535.3239098	total: 308ms	remaining: 16.8s
18:	learn: 530.1873575	total: 326ms	remaining: 16.8s
19:	

160:	learn: 345.3279275	total: 2.93s	remaining: 15.3s
161:	learn: 345.0351271	total: 2.95s	remaining: 15.3s
162:	learn: 344.7215928	total: 2.96s	remaining: 15.2s
163:	learn: 344.5203644	total: 2.98s	remaining: 15.2s
164:	learn: 344.3379870	total: 3s	remaining: 15.2s
165:	learn: 344.0467581	total: 3.02s	remaining: 15.2s
166:	learn: 343.9589862	total: 3.03s	remaining: 15.1s
167:	learn: 343.6429157	total: 3.05s	remaining: 15.1s
168:	learn: 343.5512748	total: 3.07s	remaining: 15.1s
169:	learn: 343.4787408	total: 3.08s	remaining: 15.1s
170:	learn: 343.4056775	total: 3.1s	remaining: 15s
171:	learn: 343.1856222	total: 3.12s	remaining: 15s
172:	learn: 342.7170132	total: 3.14s	remaining: 15s
173:	learn: 342.2394641	total: 3.15s	remaining: 15s
174:	learn: 341.9936291	total: 3.17s	remaining: 15s
175:	learn: 341.7892827	total: 3.19s	remaining: 14.9s
176:	learn: 341.6219411	total: 3.21s	remaining: 14.9s
177:	learn: 341.3782044	total: 3.22s	remaining: 14.9s
178:	learn: 341.0472576	total: 3.24s	remai

313:	learn: 321.7467253	total: 5.57s	remaining: 12.2s
314:	learn: 321.6211350	total: 5.59s	remaining: 12.1s
315:	learn: 321.6175884	total: 5.59s	remaining: 12.1s
316:	learn: 321.5325074	total: 5.61s	remaining: 12.1s
317:	learn: 321.3488880	total: 5.63s	remaining: 12.1s
318:	learn: 321.2747080	total: 5.64s	remaining: 12.1s
319:	learn: 321.0851403	total: 5.66s	remaining: 12s
320:	learn: 320.9170372	total: 5.68s	remaining: 12s
321:	learn: 320.9099898	total: 5.7s	remaining: 12s
322:	learn: 320.6709107	total: 5.71s	remaining: 12s
323:	learn: 320.6139310	total: 5.73s	remaining: 12s
324:	learn: 320.4967525	total: 5.75s	remaining: 11.9s
325:	learn: 320.2895671	total: 5.77s	remaining: 11.9s
326:	learn: 320.2036378	total: 5.79s	remaining: 11.9s
327:	learn: 319.9241575	total: 5.8s	remaining: 11.9s
328:	learn: 319.7171449	total: 5.82s	remaining: 11.9s
329:	learn: 319.5166557	total: 5.84s	remaining: 11.9s
330:	learn: 319.3168241	total: 5.86s	remaining: 11.8s
331:	learn: 319.1039813	total: 5.87s	rem

466:	learn: 305.7819618	total: 8.26s	remaining: 9.43s
467:	learn: 305.7653016	total: 8.28s	remaining: 9.41s
468:	learn: 305.6466276	total: 8.29s	remaining: 9.39s
469:	learn: 305.5318724	total: 8.31s	remaining: 9.37s
470:	learn: 305.3850765	total: 8.33s	remaining: 9.36s
471:	learn: 305.2956172	total: 8.35s	remaining: 9.34s
472:	learn: 305.2082308	total: 8.36s	remaining: 9.32s
473:	learn: 305.1240551	total: 8.38s	remaining: 9.3s
474:	learn: 305.0329937	total: 8.4s	remaining: 9.28s
475:	learn: 305.0054115	total: 8.42s	remaining: 9.27s
476:	learn: 304.9732789	total: 8.43s	remaining: 9.25s
477:	learn: 304.8950210	total: 8.45s	remaining: 9.23s
478:	learn: 304.7529587	total: 8.47s	remaining: 9.21s
479:	learn: 304.7228561	total: 8.49s	remaining: 9.2s
480:	learn: 304.6421200	total: 8.5s	remaining: 9.18s
481:	learn: 304.5734677	total: 8.52s	remaining: 9.16s
482:	learn: 304.3339088	total: 8.54s	remaining: 9.14s
483:	learn: 304.3190304	total: 8.56s	remaining: 9.12s
484:	learn: 304.2889152	total: 8

629:	learn: 290.7391470	total: 11.1s	remaining: 6.54s
630:	learn: 290.7102120	total: 11.2s	remaining: 6.52s
631:	learn: 290.6908258	total: 11.2s	remaining: 6.5s
632:	learn: 290.6529543	total: 11.2s	remaining: 6.49s
633:	learn: 290.4254930	total: 11.2s	remaining: 6.47s
634:	learn: 290.3783596	total: 11.2s	remaining: 6.45s
635:	learn: 290.3241883	total: 11.2s	remaining: 6.43s
636:	learn: 290.2220177	total: 11.3s	remaining: 6.42s
637:	learn: 290.1151446	total: 11.3s	remaining: 6.4s
638:	learn: 289.9668765	total: 11.3s	remaining: 6.38s
639:	learn: 289.8646981	total: 11.3s	remaining: 6.36s
640:	learn: 289.7556100	total: 11.3s	remaining: 6.34s
641:	learn: 289.6196406	total: 11.3s	remaining: 6.33s
642:	learn: 289.5740170	total: 11.4s	remaining: 6.31s
643:	learn: 289.5653438	total: 11.4s	remaining: 6.29s
644:	learn: 289.5222619	total: 11.4s	remaining: 6.27s
645:	learn: 289.4761650	total: 11.4s	remaining: 6.26s
646:	learn: 289.4726418	total: 11.4s	remaining: 6.24s
647:	learn: 289.4370824	total:

792:	learn: 279.3261183	total: 14s	remaining: 3.67s
793:	learn: 279.3083353	total: 14.1s	remaining: 3.65s
794:	learn: 279.2210679	total: 14.1s	remaining: 3.63s
795:	learn: 279.1684231	total: 14.1s	remaining: 3.61s
796:	learn: 279.1136237	total: 14.1s	remaining: 3.59s
797:	learn: 279.0403538	total: 14.1s	remaining: 3.58s
798:	learn: 278.8580562	total: 14.1s	remaining: 3.56s
799:	learn: 278.8157591	total: 14.2s	remaining: 3.54s
800:	learn: 278.7373230	total: 14.2s	remaining: 3.52s
801:	learn: 278.6499424	total: 14.2s	remaining: 3.5s
802:	learn: 278.6103718	total: 14.2s	remaining: 3.49s
803:	learn: 278.5614465	total: 14.2s	remaining: 3.47s
804:	learn: 278.4747188	total: 14.3s	remaining: 3.45s
805:	learn: 278.3666752	total: 14.3s	remaining: 3.43s
806:	learn: 278.3135341	total: 14.3s	remaining: 3.42s
807:	learn: 278.3022790	total: 14.3s	remaining: 3.4s
808:	learn: 278.2750036	total: 14.3s	remaining: 3.38s
809:	learn: 278.2132779	total: 14.3s	remaining: 3.36s
810:	learn: 278.0923777	total: 1

945:	learn: 267.7474792	total: 16.7s	remaining: 955ms
946:	learn: 267.6644419	total: 16.7s	remaining: 937ms
947:	learn: 267.5707698	total: 16.8s	remaining: 919ms
948:	learn: 267.5059898	total: 16.8s	remaining: 901ms
949:	learn: 267.3692452	total: 16.8s	remaining: 884ms
950:	learn: 267.3276086	total: 16.8s	remaining: 866ms
951:	learn: 267.2670735	total: 16.8s	remaining: 848ms
952:	learn: 267.2231809	total: 16.8s	remaining: 831ms
953:	learn: 267.1907776	total: 16.9s	remaining: 813ms
954:	learn: 267.1448760	total: 16.9s	remaining: 795ms
955:	learn: 266.9760407	total: 16.9s	remaining: 778ms
956:	learn: 266.8751122	total: 16.9s	remaining: 760ms
957:	learn: 266.7379635	total: 16.9s	remaining: 742ms
958:	learn: 266.6263118	total: 16.9s	remaining: 725ms
959:	learn: 266.5408875	total: 17s	remaining: 707ms
960:	learn: 266.5343182	total: 17s	remaining: 689ms
961:	learn: 266.3936833	total: 17s	remaining: 672ms
962:	learn: 266.3646852	total: 17s	remaining: 654ms
963:	learn: 266.2789000	total: 17s	r

In [22]:
print_metrics(y_test, predictions)

MAPE                   = 0.18014938213964057
Root Mean Square Error = 417.41028805654264
Mean Absolute Error    = 273.1118516272416
R^2                    = 0.618287021302216


In [23]:
var = model.feature_importances_
for i in range(0, len(var)):
    print(X.columns[i], var[i])


Linear Feet 31.534286889208566
Total Weight 13.240008275952135
Distance (Miles) 5.42982403732937
Target buy rate 44.62790064096278
Region Origen 2.080920715984361
Region Destino 1.3509441102024686
Combinacion regiones 1.7361153303603383


### Sin data  categorica

In [24]:
#Creamos un dataframe solo con las variables de nuestro interes
columnas_seleccionadas = ['Linear Feet', 'Total Weight', 'Distance (Miles)', 'Target buy rate', 
                          'Carrier Cost (Mixed)']
df_nuevo = df[columnas_seleccionadas]

In [25]:
#Separamos la data en test y train (20-80)
from sklearn.model_selection import train_test_split

X = df_nuevo[['Linear Feet', 'Total Weight', 'Distance (Miles)', 'Target buy rate']]
y = df_nuevo['Carrier Cost (Mixed)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
#Prueba con catboost 
from catboost import CatBoostRegressor

# Crear el modelo
model = CatBoostRegressor(depth=5, learning_rate=0.025)

# Ajustar el modelo (especificando las columnas categóricas con el parámetro cat_features)
model.fit(X_train, y_train)  # cat_features indica los índices de las variables categóricas

# Hacer predicciones
predictions = model.predict(X_test)

0:	learn: 643.6950505	total: 2.66ms	remaining: 2.65s
1:	learn: 635.7872960	total: 4.51ms	remaining: 2.25s
2:	learn: 627.8626300	total: 6.92ms	remaining: 2.3s
3:	learn: 619.9912206	total: 9.39ms	remaining: 2.34s
4:	learn: 612.8835439	total: 11.6ms	remaining: 2.31s
5:	learn: 605.4470286	total: 13.5ms	remaining: 2.24s
6:	learn: 598.1495254	total: 15.4ms	remaining: 2.19s
7:	learn: 591.2935144	total: 17.2ms	remaining: 2.13s
8:	learn: 584.5120340	total: 18.9ms	remaining: 2.08s
9:	learn: 578.2319639	total: 21.9ms	remaining: 2.16s
10:	learn: 572.9121249	total: 23.6ms	remaining: 2.12s
11:	learn: 567.1464179	total: 25.5ms	remaining: 2.1s
12:	learn: 561.1694963	total: 27.3ms	remaining: 2.07s
13:	learn: 554.9352151	total: 29.2ms	remaining: 2.05s
14:	learn: 549.6023683	total: 30.9ms	remaining: 2.03s
15:	learn: 543.9505141	total: 32.6ms	remaining: 2.01s
16:	learn: 538.9940272	total: 34.8ms	remaining: 2.01s
17:	learn: 534.0857418	total: 36.5ms	remaining: 1.99s
18:	learn: 529.1660170	total: 38.2ms	rem

163:	learn: 338.2340125	total: 310ms	remaining: 1.58s
164:	learn: 338.0464042	total: 314ms	remaining: 1.59s
165:	learn: 337.7575861	total: 316ms	remaining: 1.59s
166:	learn: 337.4572321	total: 318ms	remaining: 1.59s
167:	learn: 337.2601227	total: 320ms	remaining: 1.59s
168:	learn: 337.0387031	total: 322ms	remaining: 1.58s
169:	learn: 336.8852215	total: 324ms	remaining: 1.58s
170:	learn: 336.5629549	total: 326ms	remaining: 1.58s
171:	learn: 336.2980748	total: 328ms	remaining: 1.58s
172:	learn: 336.0268224	total: 332ms	remaining: 1.59s
173:	learn: 335.8337943	total: 334ms	remaining: 1.59s
174:	learn: 335.6064805	total: 337ms	remaining: 1.59s
175:	learn: 335.4393679	total: 340ms	remaining: 1.59s
176:	learn: 335.2987860	total: 342ms	remaining: 1.59s
177:	learn: 335.0567072	total: 344ms	remaining: 1.59s
178:	learn: 334.9066840	total: 347ms	remaining: 1.59s
179:	learn: 334.7133466	total: 349ms	remaining: 1.59s
180:	learn: 334.5149939	total: 352ms	remaining: 1.59s
181:	learn: 334.4185029	tota

324:	learn: 313.1369065	total: 631ms	remaining: 1.31s
325:	learn: 313.0379015	total: 633ms	remaining: 1.31s
326:	learn: 312.8338832	total: 635ms	remaining: 1.31s
327:	learn: 312.6679620	total: 637ms	remaining: 1.3s
328:	learn: 312.5830247	total: 639ms	remaining: 1.3s
329:	learn: 312.4811371	total: 641ms	remaining: 1.3s
330:	learn: 312.4095842	total: 643ms	remaining: 1.3s
331:	learn: 312.3056483	total: 645ms	remaining: 1.3s
332:	learn: 312.1578027	total: 647ms	remaining: 1.3s
333:	learn: 312.0455782	total: 650ms	remaining: 1.29s
334:	learn: 311.8375368	total: 652ms	remaining: 1.29s
335:	learn: 311.6816640	total: 654ms	remaining: 1.29s
336:	learn: 311.5580027	total: 656ms	remaining: 1.29s
337:	learn: 311.3856402	total: 659ms	remaining: 1.29s
338:	learn: 311.3117463	total: 661ms	remaining: 1.29s
339:	learn: 311.2500797	total: 663ms	remaining: 1.29s
340:	learn: 311.1831595	total: 665ms	remaining: 1.28s
341:	learn: 311.1193840	total: 667ms	remaining: 1.28s
342:	learn: 310.9498194	total: 669

483:	learn: 295.0057056	total: 948ms	remaining: 1.01s
484:	learn: 294.9117138	total: 951ms	remaining: 1.01s
485:	learn: 294.8481739	total: 953ms	remaining: 1.01s
486:	learn: 294.7039691	total: 955ms	remaining: 1.01s
487:	learn: 294.5552244	total: 957ms	remaining: 1s
488:	learn: 294.4612013	total: 959ms	remaining: 1s
489:	learn: 294.4160453	total: 961ms	remaining: 1s
490:	learn: 294.3171284	total: 963ms	remaining: 998ms
491:	learn: 294.1936852	total: 965ms	remaining: 997ms
492:	learn: 294.1193915	total: 968ms	remaining: 995ms
493:	learn: 294.0417987	total: 970ms	remaining: 994ms
494:	learn: 293.8855358	total: 972ms	remaining: 992ms
495:	learn: 293.8134519	total: 974ms	remaining: 989ms
496:	learn: 293.7316150	total: 976ms	remaining: 987ms
497:	learn: 293.6087717	total: 977ms	remaining: 985ms
498:	learn: 293.5465927	total: 979ms	remaining: 983ms
499:	learn: 293.4597425	total: 981ms	remaining: 981ms
500:	learn: 293.3909770	total: 984ms	remaining: 980ms
501:	learn: 293.3526962	total: 985ms	

652:	learn: 277.8259760	total: 1.27s	remaining: 673ms
653:	learn: 277.6587874	total: 1.27s	remaining: 672ms
654:	learn: 277.5090812	total: 1.27s	remaining: 670ms
655:	learn: 277.4496717	total: 1.27s	remaining: 668ms
656:	learn: 277.3016943	total: 1.27s	remaining: 666ms
657:	learn: 277.1368024	total: 1.28s	remaining: 664ms
658:	learn: 277.0793994	total: 1.28s	remaining: 662ms
659:	learn: 276.9896318	total: 1.28s	remaining: 660ms
660:	learn: 276.8831479	total: 1.28s	remaining: 659ms
661:	learn: 276.8037517	total: 1.29s	remaining: 657ms
662:	learn: 276.7245891	total: 1.29s	remaining: 655ms
663:	learn: 276.6720669	total: 1.29s	remaining: 653ms
664:	learn: 276.6069083	total: 1.29s	remaining: 651ms
665:	learn: 276.4843815	total: 1.29s	remaining: 649ms
666:	learn: 276.4027038	total: 1.29s	remaining: 647ms
667:	learn: 276.2835325	total: 1.3s	remaining: 645ms
668:	learn: 276.1519709	total: 1.3s	remaining: 643ms
669:	learn: 276.1243359	total: 1.3s	remaining: 641ms
670:	learn: 276.0864293	total: 

828:	learn: 263.3673733	total: 1.58s	remaining: 326ms
829:	learn: 263.3136128	total: 1.58s	remaining: 325ms
830:	learn: 263.2503920	total: 1.59s	remaining: 323ms
831:	learn: 263.1775807	total: 1.59s	remaining: 321ms
832:	learn: 263.0469438	total: 1.59s	remaining: 319ms
833:	learn: 262.9962989	total: 1.59s	remaining: 317ms
834:	learn: 262.9752999	total: 1.59s	remaining: 315ms
835:	learn: 262.9279928	total: 1.6s	remaining: 313ms
836:	learn: 262.8329447	total: 1.6s	remaining: 311ms
837:	learn: 262.7859314	total: 1.6s	remaining: 309ms
838:	learn: 262.7638512	total: 1.6s	remaining: 307ms
839:	learn: 262.7478394	total: 1.6s	remaining: 305ms
840:	learn: 262.6842176	total: 1.6s	remaining: 303ms
841:	learn: 262.6741524	total: 1.61s	remaining: 301ms
842:	learn: 262.5361765	total: 1.61s	remaining: 299ms
843:	learn: 262.5124909	total: 1.61s	remaining: 297ms
844:	learn: 262.4065093	total: 1.61s	remaining: 295ms
845:	learn: 262.3135496	total: 1.61s	remaining: 294ms
846:	learn: 262.2295713	total: 1.6

In [27]:
print_metrics(y_test, predictions)

MAPE                   = 0.182021249334504
Root Mean Square Error = 428.8089401423576
Mean Absolute Error    = 283.02100378951195
R^2                    = 0.5971547092018128


In [28]:
var = model.feature_importances_
for i in range(0, len(var)):
    print(X.columns[i], var[i])


Linear Feet 30.134254182378015
Total Weight 15.529963831256813
Distance (Miles) 9.569071360193972
Target buy rate 44.7667106261712


### CatBoost with Mode

In [17]:
#Creamos un dataframe solo con las variables de nuestro interes
columnas_seleccionadas = ['Mode','Linear Feet', 'Total Weight', 'Distance (Miles)', 'Target buy rate', 
                          'Cuenta_Origen_Origen', 'Cuenta_Destino_Destino', 'Combinacion estados','Carrier Cost (Mixed)', 'Load: Load Number']
df_nuevo = df[columnas_seleccionadas]

In [18]:
from sklearn.preprocessing import OneHotEncoder
# Inicializar el OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Aplicar One Hot Encoding
one_hot_encoded = encoder.fit_transform(df_nuevo[['Mode']])

# Convertir el resultado en un DataFrame
df_one_hot = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(['Mode']))

# Concatenar el One Hot Encoding con las columnas numéricas
df_final = pd.concat([df_nuevo.drop(columns=['Mode']), df_one_hot], axis=1)
df_final.head(2)

,Linear Feet,Total Weight,Distance (Miles),Target buy rate,Cuenta_Origen_Origen,Cuenta_Destino_Destino,Combinacion estados,Carrier Cost (Mixed),Load: Load Number,Mode_Partial F/SD (LTL Spot Market),Mode_Partial Van (LTL Spot Market)
0,20.5,1675.0,1081.85,2761.0,Intermedio,Florida,IntermedioFlorida,1450.0,SHP5161932,1.0,0.0
1,14.0,3800.0,1300.63,2640.0,Intermedio,Dificil,IntermedioDificil,1400.0,SHP5159992,0.0,1.0


In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df_final[['Linear Feet', 'Total Weight', 'Distance (Miles)', 'Target buy rate',
              'Mode_Partial F/SD (LTL Spot Market)', 'Mode_Partial Van (LTL Spot Market)',
              'Cuenta_Origen_Origen', 'Cuenta_Destino_Destino', 'Combinacion estados' ]]
y = df_final['Carrier Cost (Mixed)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler_X = StandardScaler()

# Escalar solo las primeras 5 columnas de X_train y X_test
X_train_scaled_part = scaler_X.fit_transform(X_train.iloc[:, :5])
X_test_scaled_part = scaler_X.transform(X_test.iloc[:, :5])

# Concatenar las columnas escaladas con el resto de las columnas no escaladas
X_train_scaled = pd.concat([pd.DataFrame(X_train_scaled_part, columns=X_train.columns[:5]),
                            X_train.iloc[:, 5:].reset_index(drop=True)], axis=1)

X_test_scaled = pd.concat([pd.DataFrame(X_test_scaled_part, columns=X_test.columns[:5]),
                           X_test.iloc[:, 5:].reset_index(drop=True)], axis=1)

In [20]:
from catboost import CatBoostRegressor

# Crear el modelo
model = CatBoostRegressor(depth=5, learning_rate=0.025)

# Ajustar el modelo (especificando las columnas categóricas con el parámetro cat_features)
model.fit(X_train, y_train, cat_features=[6,7,8])  # cat_features indica los índices de las variables categóricas

# Hacer predicciones
predictions = model.predict(X_test)

0:	learn: 643.8547406	total: 168ms	remaining: 2m 48s
1:	learn: 636.0006038	total: 191ms	remaining: 1m 35s
2:	learn: 628.6027385	total: 208ms	remaining: 1m 9s
3:	learn: 622.0561729	total: 223ms	remaining: 55.6s
4:	learn: 615.2107906	total: 240ms	remaining: 47.7s
5:	learn: 607.8072616	total: 255ms	remaining: 42.3s
6:	learn: 601.2467036	total: 272ms	remaining: 38.5s
7:	learn: 595.1789618	total: 287ms	remaining: 35.6s
8:	learn: 589.0972822	total: 304ms	remaining: 33.4s
9:	learn: 581.6579024	total: 319ms	remaining: 31.6s
10:	learn: 576.5895256	total: 335ms	remaining: 30.1s
11:	learn: 570.5610590	total: 350ms	remaining: 28.8s
12:	learn: 563.8466269	total: 371ms	remaining: 28.2s
13:	learn: 558.2515771	total: 389ms	remaining: 27.4s
14:	learn: 552.7703284	total: 405ms	remaining: 26.6s
15:	learn: 547.0678229	total: 421ms	remaining: 25.9s
16:	learn: 541.5375127	total: 440ms	remaining: 25.5s
17:	learn: 536.4105723	total: 456ms	remaining: 24.9s
18:	learn: 532.3614769	total: 473ms	remaining: 24.4s
1

160:	learn: 344.4073486	total: 2.83s	remaining: 14.8s
161:	learn: 344.3261153	total: 2.84s	remaining: 14.7s
162:	learn: 344.0069732	total: 2.86s	remaining: 14.7s
163:	learn: 343.7749947	total: 2.87s	remaining: 14.6s
164:	learn: 343.5270373	total: 2.89s	remaining: 14.6s
165:	learn: 343.2696911	total: 2.91s	remaining: 14.6s
166:	learn: 342.7811757	total: 2.92s	remaining: 14.6s
167:	learn: 342.5333662	total: 2.94s	remaining: 14.5s
168:	learn: 342.2113630	total: 2.95s	remaining: 14.5s
169:	learn: 341.9117758	total: 2.97s	remaining: 14.5s
170:	learn: 341.8189687	total: 2.98s	remaining: 14.4s
171:	learn: 341.6640526	total: 2.99s	remaining: 14.4s
172:	learn: 341.3649381	total: 3.01s	remaining: 14.4s
173:	learn: 341.1719948	total: 3.02s	remaining: 14.3s
174:	learn: 340.8452023	total: 3.04s	remaining: 14.3s
175:	learn: 340.6733507	total: 3.05s	remaining: 14.3s
176:	learn: 340.5553288	total: 3.06s	remaining: 14.2s
177:	learn: 340.4280417	total: 3.08s	remaining: 14.2s
178:	learn: 340.2325958	tota

315:	learn: 319.0146200	total: 5.24s	remaining: 11.3s
316:	learn: 318.9713362	total: 5.26s	remaining: 11.3s
317:	learn: 318.8854954	total: 5.28s	remaining: 11.3s
318:	learn: 318.8085309	total: 5.29s	remaining: 11.3s
319:	learn: 318.5817735	total: 5.31s	remaining: 11.3s
320:	learn: 318.4987496	total: 5.33s	remaining: 11.3s
321:	learn: 318.3771971	total: 5.34s	remaining: 11.3s
322:	learn: 318.2879829	total: 5.36s	remaining: 11.2s
323:	learn: 318.2300548	total: 5.38s	remaining: 11.2s
324:	learn: 318.0741552	total: 5.39s	remaining: 11.2s
325:	learn: 317.9392727	total: 5.41s	remaining: 11.2s
326:	learn: 317.8525899	total: 5.42s	remaining: 11.2s
327:	learn: 317.7855362	total: 5.43s	remaining: 11.1s
328:	learn: 317.7119975	total: 5.45s	remaining: 11.1s
329:	learn: 317.6393473	total: 5.47s	remaining: 11.1s
330:	learn: 317.5724859	total: 5.49s	remaining: 11.1s
331:	learn: 317.5075930	total: 5.5s	remaining: 11.1s
332:	learn: 317.4211785	total: 5.51s	remaining: 11s
333:	learn: 317.3599209	total: 

474:	learn: 302.9982115	total: 7.89s	remaining: 8.72s
475:	learn: 302.9482993	total: 7.91s	remaining: 8.71s
476:	learn: 302.8714909	total: 7.93s	remaining: 8.69s
477:	learn: 302.7465580	total: 7.94s	remaining: 8.67s
478:	learn: 302.6092077	total: 7.96s	remaining: 8.65s
479:	learn: 302.3670074	total: 7.97s	remaining: 8.64s
480:	learn: 302.3021150	total: 7.99s	remaining: 8.62s
481:	learn: 302.0611010	total: 8s	remaining: 8.6s
482:	learn: 302.0124770	total: 8.02s	remaining: 8.59s
483:	learn: 301.9700481	total: 8.04s	remaining: 8.57s
484:	learn: 301.6825092	total: 8.05s	remaining: 8.55s
485:	learn: 301.5845466	total: 8.07s	remaining: 8.53s
486:	learn: 301.4877395	total: 8.09s	remaining: 8.52s
487:	learn: 301.4395386	total: 8.11s	remaining: 8.51s
488:	learn: 301.3633280	total: 8.12s	remaining: 8.49s
489:	learn: 301.2387775	total: 8.14s	remaining: 8.47s
490:	learn: 301.1961764	total: 8.15s	remaining: 8.45s
491:	learn: 301.0733154	total: 8.17s	remaining: 8.44s
492:	learn: 300.8695628	total: 8

632:	learn: 288.3157366	total: 10.5s	remaining: 6.11s
633:	learn: 288.1937223	total: 10.6s	remaining: 6.1s
634:	learn: 288.1604638	total: 10.6s	remaining: 6.08s
635:	learn: 288.0803279	total: 10.6s	remaining: 6.06s
636:	learn: 287.9978836	total: 10.6s	remaining: 6.05s
637:	learn: 287.9139378	total: 10.6s	remaining: 6.03s
638:	learn: 287.8547467	total: 10.6s	remaining: 6.01s
639:	learn: 287.6146625	total: 10.7s	remaining: 5.99s
640:	learn: 287.5703523	total: 10.7s	remaining: 5.98s
641:	learn: 287.4123295	total: 10.7s	remaining: 5.96s
642:	learn: 287.3713104	total: 10.7s	remaining: 5.94s
643:	learn: 287.2593424	total: 10.7s	remaining: 5.92s
644:	learn: 287.1875004	total: 10.7s	remaining: 5.91s
645:	learn: 287.1802805	total: 10.8s	remaining: 5.89s
646:	learn: 287.0562100	total: 10.8s	remaining: 5.88s
647:	learn: 287.0052712	total: 10.8s	remaining: 5.86s
648:	learn: 286.7519428	total: 10.8s	remaining: 5.84s
649:	learn: 286.7150626	total: 10.8s	remaining: 5.82s
650:	learn: 286.6809266	total

790:	learn: 276.1258304	total: 13.2s	remaining: 3.5s
791:	learn: 276.0524643	total: 13.3s	remaining: 3.48s
792:	learn: 276.0162610	total: 13.3s	remaining: 3.46s
793:	learn: 275.9326058	total: 13.3s	remaining: 3.45s
794:	learn: 275.8568527	total: 13.3s	remaining: 3.43s
795:	learn: 275.7352031	total: 13.3s	remaining: 3.41s
796:	learn: 275.6142093	total: 13.3s	remaining: 3.4s
797:	learn: 275.5460903	total: 13.3s	remaining: 3.38s
798:	learn: 275.4631746	total: 13.4s	remaining: 3.36s
799:	learn: 275.3808859	total: 13.4s	remaining: 3.35s
800:	learn: 275.3577856	total: 13.4s	remaining: 3.33s
801:	learn: 275.2471507	total: 13.4s	remaining: 3.31s
802:	learn: 275.1888209	total: 13.4s	remaining: 3.29s
803:	learn: 275.1059005	total: 13.4s	remaining: 3.28s
804:	learn: 275.0581437	total: 13.5s	remaining: 3.26s
805:	learn: 274.9972002	total: 13.5s	remaining: 3.24s
806:	learn: 274.9400383	total: 13.5s	remaining: 3.23s
807:	learn: 274.7578055	total: 13.5s	remaining: 3.21s
808:	learn: 274.6910773	total:

954:	learn: 265.3352686	total: 15.9s	remaining: 749ms
955:	learn: 265.3211017	total: 15.9s	remaining: 733ms
956:	learn: 265.3147196	total: 15.9s	remaining: 716ms
957:	learn: 265.0987367	total: 16s	remaining: 699ms
958:	learn: 265.0761434	total: 16s	remaining: 683ms
959:	learn: 264.9843062	total: 16s	remaining: 666ms
960:	learn: 264.8043045	total: 16s	remaining: 649ms
961:	learn: 264.6969826	total: 16s	remaining: 633ms
962:	learn: 264.6021096	total: 16s	remaining: 616ms
963:	learn: 264.5414995	total: 16s	remaining: 599ms
964:	learn: 264.4432302	total: 16.1s	remaining: 583ms
965:	learn: 264.4394391	total: 16.1s	remaining: 566ms
966:	learn: 264.4077071	total: 16.1s	remaining: 549ms
967:	learn: 264.2935073	total: 16.1s	remaining: 533ms
968:	learn: 264.2743139	total: 16.1s	remaining: 516ms
969:	learn: 264.2228775	total: 16.1s	remaining: 499ms
970:	learn: 264.1841479	total: 16.2s	remaining: 483ms
971:	learn: 264.1515892	total: 16.2s	remaining: 466ms
972:	learn: 263.9744153	total: 16.2s	remai

In [21]:
print_metrics(y_test, predictions)

MAPE                   = 0.18060897691409178
Root Mean Square Error = 421.9261880297988
Mean Absolute Error    = 273.946341430989
R^2                    = 0.6099829505732911


In [23]:
X = df_final[['Linear Feet', 'Total Weight', 'Distance (Miles)', 'Target buy rate',
              'Mode_Partial F/SD (LTL Spot Market)', 'Mode_Partial Van (LTL Spot Market)',
              'Cuenta_Origen_Origen', 'Cuenta_Destino_Destino', 'Combinacion estados', 'Load: Load Number']]
y = df_final['Carrier Cost (Mixed)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
df_test = pd.concat([X_test, y_test], axis=1)
df_test.head(2)

,Linear Feet,Total Weight,Distance (Miles),Target buy rate,Mode_Partial F/SD (LTL Spot Market),Mode_Partial Van (LTL Spot Market),Cuenta_Origen_Origen,Cuenta_Destino_Destino,Combinacion estados,Load: Load Number,Carrier Cost (Mixed)
566,13.0,13685.0,1212.61,2572.0,0.0,1.0,Texas,Florida,TexasFlorida,SHP4505595,1400.0
1101,14.0,6000.0,1160.24,3311.0,1.0,0.0,Dificil,Florida,DificilFlorida,SHP3615146,1400.0


In [25]:
df_test['Predicción'] = predictions

In [26]:
df_test['diferencias'] = abs(df_test['Carrier Cost (Mixed)'] - df_test['Predicción'])
df_test = df_test.sort_values(by='diferencias', ascending= False)
df_test.head(2)

,Linear Feet,Total Weight,Distance (Miles),Target buy rate,Mode_Partial F/SD (LTL Spot Market),Mode_Partial Van (LTL Spot Market),Cuenta_Origen_Origen,Cuenta_Destino_Destino,Combinacion estados,Load: Load Number,Carrier Cost (Mixed),Predicción,diferencias
464,17.0,10050.0,3258.36,6405.0,1.0,0.0,Dificil,Florida,DificilFlorida,SHP4682096,6100.0,3054.278290,3045.721710
382,19.0,5929.0,3076.58,6346.0,1.0,0.0,Intermedio,Dificil,IntermedioDificil,SHP4788447,300.0,3215.513513,2915.513513


In [27]:
df_test.head(20)

,Linear Feet,Total Weight,Distance (Miles),Target buy rate,Mode_Partial F/SD (LTL Spot Market),Mode_Partial Van (LTL Spot Market),Cuenta_Origen_Origen,Cuenta_Destino_Destino,Combinacion estados,Load: Load Number,Carrier Cost (Mixed),Predicción,diferencias
464,17.00,10050.0,3258.36,6405.0,1.0,0.0,Dificil,Florida,DificilFlorida,SHP4682096,6100.0,3054.278290,3045.721710
382,19.00,5929.0,3076.58,6346.0,1.0,0.0,Intermedio,Dificil,IntermedioDificil,SHP4788447,300.0,3215.513513,2915.513513
1328,21.00,5232.0,1953.57,5191.0,0.0,1.0,Dificil,Florida,DificilFlorida,SHP3239920,4500.0,2950.068071,1549.931929
1252,14.00,2796.0,1959.07,5025.0,0.0,1.0,Dificil,Florida,DificilFlorida,SHP3379875,3700.0,2246.656241,1453.343759
1074,16.00,6247.0,1537.42,4000.0,0.0,1.0,Texas,Florida,TexasFlorida,SHP3677093,3210.0,1875.696000,1334.304000
571,22.50,10235.0,1539.32,3937.0,0.0,1.0,Intermedio,Florida,IntermedioFlorida,SHP4495658,1400.0,2544.340077,1144.340077
316,18.67,10600.0,2783.00,5818.0,1.0,0.0,Intermedio,Dificil,IntermedioDificil,SHP4852780,3700.0,2569.381881,1130.618119
1030,20.01,12872.0,3120.18,6075.0,0.0,1.0,California,Dificil,CaliforniaDificil,SHP3778746,2167.5,3207.802799,1040.302799
918,10.00,5000.0,1686.30,3245.0,1.0,0.0,Texas,Dificil,TexasDificil,SHP3990908,2200.0,1268.686199,931.313801
1296,24.00,3492.0,2554.09,5376.0,0.0,1.0,Intermedio,Florida,IntermedioFlorida,SHP3298806,4000.0,3096.650590,903.349410
